In [1]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

In [13]:
''' #read_data = pd.read_csv('sample_tickers.csv')
read_data = ['USA.TO', 'ACB.TO']

#create start and end date based on dynamic data
start_date = "2024-09-01"
end_date = "2024-11-01"
djia = yf.Ticker('DJIA') #use any other market index as well
market_hist = djia.history(start=start_date, end=end_date)
market_pd = market_hist['Close']

beta_df = pd.DataFrame()
expected_market = market_pd.mean()
for i in read_data:
    ticker = yf.Ticker(i)
    ticker_hist = ticker.history(start=start_date, end=end_date)
    ticker_pd = ticker_hist['Close']
    expected_asset = ticker_pd.mean()
    beta = expected_asset.cov(expected_market) / expected_market.var()
    beta_df[i] = beta

beta_df.transpose()
beta_df.columns=['ticker', 'price']
beta_df = beta_df.sort_values(beta_df.price, ascending = False)
print(beta_df)


#use slicing for lower and upper bound picking '''

' #read_data = pd.read_csv(\'sample_tickers.csv\')\nread_data = [\'USA.TO\', \'ACB.TO\']\n\n#create start and end date based on dynamic data\nstart_date = "2024-09-01"\nend_date = "2024-11-01"\ndjia = yf.Ticker(\'DJIA\') #use any other market index as well\nmarket_hist = djia.history(start=start_date, end=end_date)\nmarket_pd = market_hist[\'Close\']\n\nbeta_df = pd.DataFrame()\nexpected_market = market_pd.mean()\nfor i in read_data:\n    ticker = yf.Ticker(i)\n    ticker_hist = ticker.history(start=start_date, end=end_date)\n    ticker_pd = ticker_hist[\'Close\']\n    expected_asset = ticker_pd.mean()\n    beta = expected_asset.cov(expected_market) / expected_market.var()\n    beta_df[i] = beta\n\nbeta_df.transpose()\nbeta_df.columns=[\'ticker\', \'price\']\nbeta_df = beta_df.sort_values(beta_df.price, ascending = False)\nprint(beta_df)\n\n\n#use slicing for lower and upper bound picking '


Since we have chosen the market beat strategy, we want to maximize our returns. We also want minimum diversification. 

Beta calculation:
Beta value measures a stock's volatility and sensitivity to market movements relative to an index. A Beta greater than 1 means the stock tends to have higher volatility, while a Beta less than 1 indicates that the stock is less volatile than the market. 

We get a list of the stocks that have the highest beta. This gives us a portfolio that is positively correlated and volatile compared to the market. This also implies that the portfolio has the least market diversification and the highest volume of change. 

In [ ]:
start_date = "2024-09-01"
end_date = "2024-11-01"

MarketIndex1 = "^GSPTSE"  # TSX 60 #confirm ticker
MarketIndex2 = "^GSPC"  # S&P 500


market_hist1 = yf.Ticker(MarketIndex1).history(start=start_date, end=end_date)['Close']
market_hist2 = yf.Ticker(MarketIndex2).history(start=start_date, end=end_date)['Close']

market_hist1["Returns"] = market_hist1.pct_change() * 100
market_hist2["Returns"] = market_hist2.pct_change() * 100

market_avg_returns = (market_hist1["Returns"] + market_hist2["Returns"]) / 2

market_var = market_avg_returns.var()

# get_stock_beta returns the beta of the ticker inputted
def beta_val(stock_returns):
    df = pd.DataFrame({"Market Returns": market_avg_returns, "Stock Returns": stock_returns}).dropna()
    covariance = df.cov().iloc[0, 1]
    return covariance / market_var


#betas_list takes a list of tickers and outputs a list of all the betas of every ticker in the list
def betas_list(ticker_list):
    betas = {}
    for ticker in ticker_list:
        stock_hist = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
        ticker_returns = stock_hist.pct_change()*100 
        betas[ticker] = beta_val(ticker_returns)
    return betas


ticker_list = ["AAPL", "MSFT", "GOOGL"]
all_betas = betas_list(ticker_list)

print("Betas:", all_betas)

Betas: {'AAPL': 1.1112476004934124, 'MSFT': 1.6016277866245967, 'GOOGL': 1.0402340367516663}


In [10]:
#Creates a dataframe with keys being tickers and corresponding to their values, the ticker's beta
filtered = ["AAPL", "MSFT", "GOOGL"] #example
beta_dict = betas_list(filtered)  #change based on taken
beta_df = pd.DataFrame.from_dict(beta_dict, orient='index')
beta_df.columns = ['Beta']
beta_df.index.name = 'Ticker'

In [11]:
#Sorts the dictionary from lowest beta to highest beta
sorted_beta_df = beta_df.sort_values('Beta').copy()

#Creates a dataframe with the 10 highest betas
top_twelve = sorted_beta_df.iloc[-12:]

Sharpe Ratio:
Sharpe Ratio measures the return of an investment relative to its risk. A higher Sharpe ratio indicates a better risk-adjusted return, meaning the stock or portfolio is delivering more reward for the risk taken. It helps identify stocks that offer the best potential for growth, with minimal unnecessary risk.

In [12]:

tickers = ["AAPL", "MSFT", "GOOGL"]  # Example list of tickers
risk_free_rate = 0  # Example 

# Function to calculate the Sharpe Ratio
def calculate_sharpe_ratio(ticker, risk_free_rate):
    stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
    stock_returns = stock_data.pct_change()
    average_return = stock_returns.mean()
    std_deviation = stock_returns.std()
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation
    return sharpe_ratio


sharpe_ratios = {}
for ticker in tickers:
    sharpe_ratios[ticker] = calculate_sharpe_ratio(ticker, risk_free_rate)

print("Sharpe Ratios:")
for ticker, sharpe_ratio in sharpe_ratios.items():
    print(ticker,sharpe_ratio)


Sharpe Ratios:
AAPL 0.031480065526698284
MSFT -0.00625280357779384
GOOGL 0.1609000529527446


## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat or Market Meet)

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.